In [236]:
import numpy as np
import pandas as pd
import seaborn as sns

In [237]:
df=sns.load_dataset('tips')

In [238]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [239]:
X,y=df.drop(columns=['time'],axis=1),df['time']

In [240]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [241]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=99)

In [242]:
X_train.head()

,total_bill,tip,sex,smoker,day,size
214,28.17,6.50,Female,Yes,Sat,3
36,16.31,2.00,Male,No,Sat,3
54,25.56,4.34,Male,No,Sun,4
86,13.03,2.00,Male,No,Thur,2
115,17.31,3.50,Female,No,Sun,2


In [243]:
#automate preprocessing - data cleanig and feature transformation

In [244]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [245]:
categorical_cols=['sex','smoker','day']
numerical_cols=['total_bill','tip','size']

In [246]:
num_pipeline=Pipeline(
    steps=[
        ('impute',SimpleImputer(strategy='mean')),
        ('scaler',StandardScaler())
    ]
)

cat_pipeline=Pipeline(
    steps=[
        ('impute',SimpleImputer(strategy='most_frequent')),
        ('encoder',OneHotEncoder())
    ]
)

preprocessor=ColumnTransformer(
    [
        ('num_pipeline',num_pipeline,numerical_cols),
        ('cat_pipeline',cat_pipeline,categorical_cols)
    ]
)

In [247]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.fit_transform(X_test)

In [248]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
#automate model training and evaluation
models={
    'RandomForest':RandomForestClassifier(),
    'DecitionTree':DecisionTreeClassifier(),
    'SVC':SVC()
}

def evaluate_model(X_train,X_test,y_train,y_test,models):
    report = {}
    for i in range(len(models)):
        model=list(models.values())[i]
        model.fit(X_train,y_train)
        y_pred=model.predict(X_test)
        score=accuracy_score(y_test,y_pred)
        report[list(models.values())[i]]=score
    return report

In [249]:
evaluate_model(X_train,X_test,y_train,y_test,models)

{RandomForestClassifier(): 0.9591836734693877,
 DecisionTreeClassifier(): 0.9795918367346939,
 SVC(): 0.9795918367346939}

In [251]:
## Hypeparameter Tuning
classifier=RandomForestClassifier()

params={'max_depth':[3,5,10,None],
              'n_estimators':[100,200,300],
               'criterion':['gini','entropy']
              }

In [252]:
from sklearn.model_selection import RandomizedSearchCV

In [253]:
cv=RandomizedSearchCV(classifier,param_distributions=params,scoring='accuracy',cv=5,verbose=3)
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=0.974 total time=   0.2s
[CV 2/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=0.949 total time=   0.2s
[CV 3/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=0.974 total time=   0.2s
[CV 4/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=1.000 total time=   0.2s
[CV 5/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=0.923 total time=   0.2s
[CV 1/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.974 total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.949 total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.974 total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=5, n_estimators=100;, score=1.000 total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.923 total time=   0.0s

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [254]:
cv.best_params_

{'n_estimators': 300, 'max_depth': 10, 'criterion': 'entropy'}